# Import Libraries

In [1]:
import time
from datasets import load_from_disk
import pandas as pd
from sentence_transformers import SentenceTransformer
from sentence_transformers.quantization import quantize_embeddings
import faiss
from usearch.index import Index
import numpy as np
import os

/home/swastikm/work/senetence-transformer-in-action-main/semantic_search_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Dataset

In [2]:
base_path = os.getcwd()
full_path = os.path.join(base_path, 'conala')

In [3]:
conala_dataset = load_from_disk(full_path)

```Text
Load the int8 and binary indices. Int8 is loaded as a view to save memory, as we never actually perform search with it.
Int8 embedding is required to perform rescoring of fetched document. Rescoring is done by performing inner product with F32 embedding of Query
```
[Efficient Passage Retrieval with Hashing for Open-domain Question Answering](https://arxiv.org/abs/2106.00882)

In [4]:
# Load the int8 and binary indices. Int8 is loaded as a view to save memory, as we never actually perform search with it.
int8_view = Index.restore(os.path.join(base_path, 'conala_int8_usearch.index'), view=True)
binary_index: faiss.IndexBinaryFlat = faiss.read_index_binary(os.path.join(base_path, 'conala.index'))

# Import Model to generate embedding

In [5]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

/home/swastikm/work/senetence-transformer-in-action-main/semantic_search_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
def search(query, top_k: int = 20):
    # 1. Embed the query as float32
    query_embedding = model.encode(query)

    # 2. Quantize the query to ubinary. To perform actual search with faiss
    query_embedding_ubinary = quantize_embeddings(query_embedding.reshape(1, -1), "ubinary")


    # 3. Search the binary index 
    index =  binary_index
    _scores, binary_ids = index.search(query_embedding_ubinary, top_k)
    binary_ids = binary_ids[0]


    # 4. Load the corresponding int8 embeddings. To perform rescoring to calculate score of fetched documents.
    int8_embeddings = int8_view[binary_ids].astype(int)

    # 5. Rescore the top_k * rescore_multiplier using the float32 query embedding and the int8 document embeddings
    scores = query_embedding @ int8_embeddings.T

    # 6. Sort the scores and return the top_k
    start_time = time.time()
    indices = scores.argsort()[::-1][:top_k]
    top_k_indices = binary_ids[indices]
    top_k_scores = scores[indices]

    top_k_codes = conala_dataset[top_k_indices]

    return top_k_codes


In [7]:
user_prompt = input('Enter python coding query')
top_k_outputs = search(user_prompt)

In [8]:
probs = top_k_outputs['prob']
snippets = top_k_outputs['snippet']

In [9]:
idx = np.argsort(probs)[::-1]
results = np.array(snippets)[idx]

In [10]:
filtered_results = []
for item in results:
    if len(filtered_results)<3:
        if item not in filtered_results:
            filtered_results.append(item)
filtered_results

['[x[1] for x in elements]',
 'map(itemgetter(1), elements)',
 'zip(*elements)[1]']

In [11]:
output_template = "User Query: {user_query}\nBelow are some examples of previous conversations.\nQuery: {query1} Solution: {solution1}\nQuery: {query2} Solution: {solution2}\nYou may use the above examples for reference only. Create your own solution and provide only the solution"

In [12]:
output_template = "The top three most relevant code snippets from the database are:\n\n1. {snippet1}\n\n2. {snippet2}\n\n3. {snippet3}"

In [13]:
output = f'{output_template.format(snippet1=filtered_results[0],snippet2=filtered_results[1],snippet3=filtered_results[2])}'

In [14]:
print(output)

The top three most relevant code snippets from the database are:

1. [x[1] for x in elements]

2. map(itemgetter(1), elements)

3. zip(*elements)[1]
